In [14]:
import os
import cv2
import time
import joblib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, LSTM, Dense, Dropout, MaxPool2D, Flatten

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score

tf.__version__

'2.4.1'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

# BASE_DIR       = '../'
BASE_DIR       = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'
IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 40
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN    = 1120
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.0001
DECAY          = 0.0

CONFIG         = '_L_7_S_40x40_E_7'

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [4]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 1))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 1))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = np.expand_dims(resized, axis=-1)
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = np.expand_dims(resized, axis=-1)
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test


def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, 1))
    y = np.zeros((DATASET_LEN, 1))
    
    count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            resized = cv2.resize(img, IMG_SIZE)
            X[count, :] = np.expand_dims(resized, axis=-1)
            y[count, 0] = GESTURES.index(gesture)
            count = count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_test = X[((test_user - 1) * TEST_LEN) : (test_user * TEST_LEN), :, :, :]
    y_test = y[((test_user - 1) * TEST_LEN) : (test_user * TEST_LEN), :]
    X_train = np.delete(X, np.arange((test_user - 1) * TEST_LEN, test_user * TEST_LEN), axis=0)
    y_train = np.delete(y, np.arange((test_user - 1) * TEST_LEN, test_user * TEST_LEN), axis=0)

    print('√')

    return X_train, X_test, y_train, y_test

In [5]:
# LOAD AND SAVE DATA
# load_and_save_data('XY')
# load_and_save_data('YZ')
# load_and_save_data('ZX')

In [6]:
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

In [5]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data_from_joblib('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data_from_joblib('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data_from_joblib('ZX')

Loading data for XY plane ... √
Loading data for YZ plane ... √
Loading data for ZX plane ... √


In [28]:
IMG_SHAPE = IMG_SIZE + (1,)
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.0, offset= 0, input_shape=IMG_SHAPE)

In [32]:
def get_model():
    model = Sequential()

    model.add(rescale)

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(16))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model

In [33]:
model_xy = get_model()
X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 1s 13ms/step - loss: 2.7496 - accuracy: 0.0708 - val_loss: 3.3731 - val_accuracy: 0.0000e+00
Epoch 2/7
30/30 [==============================] - 0s 8ms/step - loss: 2.6811 - accuracy: 0.1125 - val_loss: 3.5658 - val_accuracy: 0.0000e+00
Epoch 3/7
30/30 [==============================] - 0s 8ms/step - loss: 2.6561 - accuracy: 0.0957 - val_loss: 3.5281 - val_accuracy: 0.0000e+00
Epoch 4/7
30/30 [==============================] - 0s 8ms/step - loss: 2.6113 - accuracy: 0.1477 - val_loss: 3.5042 - val_accuracy: 0.0000e+00
Epoch 5/7
30/30 [==============================] - 0s 8ms/step - loss: 2.4848 - accuracy: 0.1881 - val_loss: 3.8530 - val_accuracy: 0.0000e+00
Epoch 6/7
30/30 [==============================] - 0s 8ms/step - loss: 2.3406 - accuracy: 0.2269 - val_loss: 3.8742 - val_accuracy: 0.0000e+00
Epoch 7/7
30/30 [==============================] - 0s 8ms/step - loss: 2.1990 - accuracy: 0.2701 - val_loss: 4.2837 - val_accuracy: 0.0000e+0

In [ ]:
# prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# y_pred_xy = prob_xy.predict(X_test_xy)
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
keras.backend.clear_session()

In [ ]:
model_yz = get_model()
X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

In [ ]:
# prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# y_pred_yz = prob_yz.predict(X_test_yz)
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

In [ ]:
model_zx = get_model()
X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

In [ ]:
# prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# y_pred_zx = prob_zx.predict(X_test_zx)
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

In [ ]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

In [ ]:
config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_splstm_bw' + CONFIG + '.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()